<a href="https://colab.research.google.com/github/Monisha-11/monte-carlo-control/blob/main/RLEXP_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-c6gqtnk_/gym-walk_239ccafe2f4e419e85f499737d9acc8b
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-c6gqtnk_/gym-walk_239ccafe2f4e419e85f499737d9acc8b
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done


In [14]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

# Reference https://github.com/mimoralea/gym-walk

In [15]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [16]:

def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [17]:
def probability_success(env, pi, goal_state, n_episodes=150000, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [18]:
def mean_return(env, pi, n_episodes=150000, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)



        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [19]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
#LEFT, RIGHT = range(2)

In [20]:
def decay_schedule(init_value,min_value,decay_ratio,max_steps,log_start=-2,log_base=10):
  decay_steps=int(max_steps*decay_ratio)
  rem_steps=max_steps - decay_steps

  values=np.logspace(log_start,0,decay_steps,base=log_base,endpoint=True)[::-1]
  values=(values -  values.min()) /(values.max()-values.min())
  values=(init_value - min_value)*values+min_value
  values=np.pad(values,(0,rem_steps),'edge')
  return values

In [21]:
from itertools import count
def generate_trajectory(select_action,Q,epsilon,env,max_steps=200):
  done,trajectory = False,[]
  while not done:
    state = env.reset()
    for t in count():
      action = select_action(state,Q,epsilon)
      next_state,reward,done,_=env.step(action)
      experience = (state,action,reward,next_state,done)
      trajectory.append(experience)
      if done:
        break
      if t>=max_steps-1:
        trajectory = []
        break
      state = next_state
  return np.array(trajectory,np.object)

In [22]:
from tqdm import tqdm
def mc_control(env, gamma = 1.0, init_alpha = 0.5, min_alpha = 0.01,
               alpha_decay_ratio = 0.5, init_epsilon = 1.0, min_epsilon = 0.1,
               epsilon_decay_ratio = 0.9, n_episodes = 3000, max_steps = 200,
               first_visit = True):
  nS, nA = env.observation_space.n, env.action_space.n

  discounts = np.logspace(0,max_steps, num=max_steps,
                          base=gamma, endpoint = False)

  alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)

  epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

  pi_track = []

  Q = np.zeros((nS, nA), dtype = np.float64)
  Q_track = np.zeros((n_episodes, nS, nA), dtype = np.float64)

  select_action = lambda state, Q, epsilon: np.argmax(Q[state]) if np.random.random() > epsilon_decay_ratio else np.random.randint(len(Q[state]))

  for e in tqdm(range(n_episodes), leave = False):
    trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)
    visited = np.zeros((nS, nA), dtype = np.bool)
    for t, (state, action, reward, _, _) in enumerate(trajectory):
      if visited[state][action] and first_visit:
        continue
      visited[state][action] = True
      n_steps = len(trajectory[t:])
      G = np.sum(discounts[:n_steps] * trajectory[t:, 2])
      Q[state][action] += alphas[e] * (G - Q[state][action])
    Q_track[e] = Q
    pi_track.append(np.argmax(Q, axis = 1))
  V = np.max(Q, axis = 1)
  pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis = 1))}[s]
  return Q, V, pi

In [23]:
Q,V,pi = mc_control (env,n_episodes=150000)
print_state_value_function(Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(pi, P)

Action-value function:
| 00 [0.02 0.01 0.01 0.01] | 01 [0.01 0.01 0.02 0.02] | 02 [0.01 0.03 0.04 0.03] | 03 [0.01 0.02 0.01 0.02] |
| 04 [0.02 0.02 0.   0.  ] |           | 06 [0.06 0.05 0.06 0.01] |           |
| 08 [0.02 0.01 0.04 0.05] | 09 [0.09 0.12 0.09 0.04] | 10 [0.15 0.17 0.17 0.06] |           |
|           | 13 [0.1  0.24 0.25 0.18] | 14 [0.28 0.53 0.51 0.45] |           |
State-value function:
| 00   0.02 | 01   0.02 | 02   0.04 | 03   0.02 |
| 04   0.02 |           | 06   0.06 |           |
| 08   0.05 | 09   0.12 | 10   0.17 |           |
|           | 13   0.25 | 14   0.53 |           |
Policy:
| 00      < | 01      ^ | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      > |           |
|           | 13      > | 14      v |           |


In [24]:
# Find the probability of success and the mean return of the first policy
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi, goal_state=goal_state)*100,
    mean_return(env, pi)))

Reaches goal 48.97%. Obtains an average undiscounted return of 0.4897.
